## Climate Change Dataset of Laws and Policies

In [1]:
from flashtext import KeywordProcessor
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

terms = ['agriculture','agricultural','agritech','agtech']
print('terms to use: {}'.format(len(terms)))
data_laws = pd.read_csv('datasets/laws_and_policies_06072020.csv')
data_laws.head(4)

terms to use: 4


,Title,Type,Geography,Geography ISO,Frameworks,Responses,Instruments,Document Types,Natural Hazards,Keywords,Sectors,Events,Documents,Parent Legislation,Description
0,Energy Sector Strategy 1387-1391 (2007/8-2012/3),executive,Afghanistan,AFG,NaN,NaN,NaN,Strategy,NaN,Energy Supply,Energy,25/12/2008|Law passed,Full text|https://climate-laws.org/rails/activ...,NaN,This strategy sets the vision and goals for th...
1,Rural Renewable Energy Policy (RREP),executive,Afghanistan,AFG,NaN,Adaptation,NaN,Policy,NaN,"Adaptation, Energy Supply",Energy,25/12/2013|Law passed,NaN,NaN,"This policy aims at creating better social, ec..."
2,National Forestry Management Policy (NFMP),executive,Afghanistan,AFG,NaN,NaN,NaN,Policy,NaN,Research And Development,NaN,25/12/2007|Law passed,NaN,NaN,NaN
3,Strategic National Action Plan for Disaster Ri...,executive,Afghanistan,AFG,DRM/DRR,"Adaptation, Disaster Risk Management",Designing processes|Governance and planning;De...,Plan,"Earthquake, Flood, Snow melt, Glacial Melt, Dr...",Adaptation,NaN,25/12/2011|Law passed||,NaN,NaN,<div>This Action Plan includes the objective o...


---
## Tagging CCM Laws and Policies related to Agriculture and LULUCF 
LULUCF = land use, land use change and forestry

In [2]:
def extract(vec, dictionary, info=False):
    matrix = []
    for line in vec:
        matrix.append(dictionary.extract_keywords(str(line), span_info=info))
    return matrix

In [3]:
data_laws['sector_agriculture'] = data_laws.Sectors.apply(lambda x: (1 if 'Agriculture' in str(x) else 0))
print(data_laws.sector_agriculture.sum())

data_laws['sector_LULUCF'] = data_laws.Sectors.apply(lambda x: (1 if 'LULUCF' in str(x) else 0))
print(data_laws.sector_LULUCF.sum())

import numpy as np
data_laws['sector_agriculture_LULUCF'] = np.where((data_laws['sector_LULUCF']==1) | (data_laws['sector_agriculture']==1), 1,0)
print(data_laws.sector_agriculture_LULUCF.sum())

241
359
472


In [4]:
terms_dict = KeywordProcessor()
terms_dict.add_keywords_from_list(terms)
terms_extracted = extract(data_laws.Description, terms_dict)
rows = [list(set(i)) if len(i)>0 else '' for i in terms_extracted]
data_laws['matches'] = [str(i).replace('[', '').replace(']', '') for i in rows]
data_laws['count_matches'] = [len(i) for i in terms_extracted]

---
## Dummy for EU countries

In [5]:
# selecting the relevant countries
# data_laws.Geography.unique()
EU=['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','European Union','Finland', 'France','Germany','Greece','Hungary', 'Ireland', 'Italy','Latvia','Lithuania','Luxembourg','Malta', 'Netherlands','Poland', 'Portugal','Romania','Slovakia', 'Slovenia', 'Spain', 'Sweden','United Kingdom',]

# creating dummy for EU country (individual countries + European Union)
EU_country= []
for i in data_laws.Geography:
    country =  1 if i in EU else 0
    EU_country.append(country)
    
# adding the dummy to the original dataset
data_laws['EU_country']=EU_country
data_laws.EU_country.sum()

433

---
## Dummy for year as from 2004

In [6]:
#  extracting the first year from the Events column

data_laws[['First','Second','Third','Fourth']]=data_laws.Events.str.split(";",expand=True)

data_laws['First_date']=data_laws['First'].str.slice(stop=10)
# data_laws['First_status']=data_laws['First'].str.slice(start=11)
data_laws['First_year']=data_laws['First_date'].str.slice(start=6)

# remove laws and policies that do not have a date indication
data_laws = data_laws[data_laws['First_year'].notna()]

# change year to numeric column
data_laws["First_year"] = pd.to_numeric(data_laws["First_year"],downcast='integer')
# data_laws["First_year"] = data_laws["First_year"].astype(int)

In [7]:
data_laws.First_year.unique()

array([2008, 2013, 2007, 2011, 2009, 2014, 2016, 2012, 2015, 2003, 2017,
       2010, 2006, 2005, 2004, 1999, 2018, 1957, 1998, 2019, 1991, 2000,
       2002, 1993, 1997, 2001, 1996, 1992, 1987, 1984, 1994, 1990, 1995,
       2020, 1976, 1963, 1989, 1960, 1980, 1979, 1947, 1948, 1961, 1982,
       1986, 1983, 1985, 1988, 1972, 1964, 1977], dtype=int16)

In [8]:
data_laws.loc[data_laws['First_year']== 'NA']

,Title,Type,Geography,Geography ISO,Frameworks,Responses,Instruments,Document Types,Natural Hazards,Keywords,...,sector_agriculture_LULUCF,matches,count_matches,EU_country,First,Second,Third,Fourth,First_date,First_year


In [9]:
# creating dummy for all laws passed starting with the last big join of countries in 2004

After_2004= []
for i in data_laws.First_year:
    year =  1 if i >2003 else 0
    After_2004.append(year)

# adding the dummy to the original dataset
data_laws['After_2004']=After_2004
data_laws.After_2004.sum()

1622

---
## Save as Excel file

In [10]:
data_laws.sort_values('count_matches', ascending=False).to_excel('datasets/laws_and_policies_AGRI.xlsx')
data_laws.head()

,Title,Type,Geography,Geography ISO,Frameworks,Responses,Instruments,Document Types,Natural Hazards,Keywords,...,matches,count_matches,EU_country,First,Second,Third,Fourth,First_date,First_year,After_2004
0,Energy Sector Strategy 1387-1391 (2007/8-2012/3),executive,Afghanistan,AFG,NaN,NaN,NaN,Strategy,NaN,Energy Supply,...,,0,0,25/12/2008|Law passed,None,None,None,25/12/2008,2008,1
1,Rural Renewable Energy Policy (RREP),executive,Afghanistan,AFG,NaN,Adaptation,NaN,Policy,NaN,"Adaptation, Energy Supply",...,,0,0,25/12/2013|Law passed,None,None,None,25/12/2013,2013,1
2,National Forestry Management Policy (NFMP),executive,Afghanistan,AFG,NaN,NaN,NaN,Policy,NaN,Research And Development,...,,0,0,25/12/2007|Law passed,None,None,None,25/12/2007,2007,1
3,Strategic National Action Plan for Disaster Ri...,executive,Afghanistan,AFG,DRM/DRR,"Adaptation, Disaster Risk Management",Designing processes|Governance and planning;De...,Plan,"Earthquake, Flood, Snow melt, Glacial Melt, Dr...",Adaptation,...,,0,0,25/12/2011|Law passed||,None,None,None,25/12/2011,2011,1
4,The National Environmental Action Plan (NEAP),executive,Afghanistan,AFG,NaN,Adaptation,NaN,Plan,NaN,Adaptation,...,,0,0,25/12/2009|Law passed,None,None,None,25/12/2009,2009,1
